In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import numpy as np

# Load VGG16 pre-trained on ImageNet without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of the base_model
for layer in base_model.layers:
    layer.trainable = False

# Correctly reference tf.keras.layers.GlobalAveragePooling2D
model = Model(inputs=base_model.input, outputs=tf.keras.layers.GlobalAveragePooling2D()(base_model.output))

# Continue with your data generators and the rest of your code...


# Prepare your data generators
datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory(
    'dataset\Archive',
    target_size=(150, 150),
    batch_size=32,
    class_mode=None,  # No labels needed
    shuffle=False)

# Use model to predict and extract features
features = model.predict(generator, steps=np.ceil(generator.samples/generator.batch_size))

# Get labels associated with each feature
labels = generator.classes


Found 42500 images belonging to 13 classes.
   2/1329 [..............................] - ETA: 48:44  

C:\Users\vinit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  25/1329 [..............................] - ETA: 48:06

C:\Users\vinit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (96714256 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


1117/1329 [========================>.....] - ETA: 15:47

C:\Users\vinit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


1329/1329 [==============================] - 6077s 5s/step


In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train k-NN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Evaluate the classifier
predictions = knn.predict(X_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.23      0.34      0.28       276
           1       0.52      0.51      0.51       646
           2       0.36      0.62      0.46      1081
           3       0.46      0.31      0.37       506
           4       0.61      0.55      0.58       448
           5       0.50      0.59      0.54       594
           6       0.64      0.26      0.37       269
           7       0.43      0.41      0.42      1088
           8       0.50      0.46      0.48      1204
           9       0.37      0.34      0.36       498
          10       0.51      0.47      0.49      1352
          11       0.34      0.07      0.12       318
          12       0.68      0.41      0.51       220

    accuracy                           0.45      8500
   macro avg       0.47      0.41      0.42      8500
weighted avg       0.47      0.45      0.45      8500

